<a href="https://colab.research.google.com/github/Sumitkevlani/LLM/blob/main/Equity_Research_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install langchain langchain-core langchain-huggingface langchain-community faiss-cpu sentence-transformers langchain-text-splitters unstructured --quiet

In [3]:
from huggingface_hub import login
login()

In [68]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    max_new_tokens=64,
    do_sample=False,
    repetition_penalty=1.03,
)

In [5]:
from langchain_community.document_loaders import UnstructuredURLLoader

In [8]:
loader = UnstructuredURLLoader(
    urls=["https://en.wikipedia.org/wiki/Indian_Rebellion_of_1857","https://en.wikipedia.org/wiki/Jallianwala_Bagh_massacre"]
)

In [54]:
data = loader.load()

In [55]:
print(data)

[Document(page_content='Toggle the table of contents\n\nIndian Rebellion of 1857\n\n75 languages\n\nالعربية\n\nԱրեւմտահայերէն\n\nঅসমীয়া\n\nAsturianu\n\nAzərbaycanca\n\nتۆرکجه\n\nবাংলা\n\n閩南語 / Bân-lâm-gú\n\nБашҡортса\n\nБеларуская\n\nभोजपुरी\n\nБългарски\n\nBrezhoneg\n\nCatalà\n\nČeština\n\nCymraeg\n\nDansk\n\nDeutsch\n\nΕλληνικά\n\nEspañol\n\nEsperanto\n\nEuskara\n\nفارسی\n\nFrançais\n\nGaeilge\n\nગુજરાતી\n\n한국어\n\nՀայերեն\n\nहिन्दी\n\nHrvatski\n\nIdo\n\nBahasa Indonesia\n\nItaliano\n\nעברית\n\nಕನ್ನಡ\n\nქართული\n\nLatviešu\n\nMagyar\n\nमैथिली\n\nമലയാളം\n\nमराठी\n\nBahasa Melayu\n\nNederlands\n\nनेपाली\n\n日本語\n\nNorsk bokmål\n\nNorsk nynorsk\n\nOccitan\n\nଓଡ଼ିଆ\n\nOʻzbekcha / ўзбекча\n\nਪੰਜਾਬੀ\n\nپنجابی\n\nپښتو\n\nPolski\n\nPortuguês\n\nRomână\n\nРусский\n\nसंस्कृतम्\n\nShqip\n\nSimple English\n\nسنڌي\n\nСрпски / srpski\n\nSuomi\n\nSvenska\n\nTagalog\n\nதமிழ்\n\nతెలుగు\n\nไทย\n\nTürkçe\n\nУкраїнська\n\nاردو\n\nTiếng Việt\n\n吴语\n\n粵語\n\n中文\n\nEdit links\n\nArticle\n\nTalk\n\nEnglish\n\

In [56]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [57]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
all_splits = text_splitter.split_documents(documents=data)

In [58]:
from langchain_huggingface import HuggingFaceEmbeddings

In [59]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [60]:
from langchain.vectorstores.faiss import FAISS
vectorstore = FAISS.from_documents(all_splits, hf)

In [62]:
import pickle

with open("new_vectorstore.pkl", "wb") as f:
  pickle.dump(vectorstore, f)


In [63]:
import os
file_path = "new_vectorstore.pkl"
if os.path.exists(file_path):
  with open(file_path, "rb") as f:
    vectorIndex = pickle.load(f)

In [64]:
from langchain.chains import RetrievalQAWithSourcesChain

In [69]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm,retriever=vectorIndex.as_retriever())

In [70]:
chain

RetrievalQAWithSourcesChain(combine_documents_chain=MapReduceDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-v0.1', max_new_tokens=64, repetition_penalty=1.03, model='mistralai/Mistral-7B-v0.1', client=<InferenceClient(model='mistralai/Mistral-7B-v0.1', timeout=120)>, async_client=<InferenceClient(model='mistralai/Mistral-7B-v0.1', timeout=120)>, task='text-generation')), reduce_documents_chain=ReduceDocumentsChain(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'summaries'], template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf

In [73]:
query = "Who was the 81-year old Mughal Ruler?"

chain({"question": query},return_only_outputs=True)

{'answer': ' Bahadur Shah Zafar\n', 'sources': ''}